<a href="https://colab.research.google.com/github/ItzWarLockOp/Google-Colab-Scriptz/blob/main/Torrent2Gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center> <img src="https://i.ibb.co/5GDWzfV/Picsart-23-04-29-15-39-46-742.png" height=40px class="icon" alt="icon">

# <b><center> Torrent to Gdrive

___________________________________________

# <b> <center> Mount Gdrive   <img src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" height="20px" alt="Icon"> 
# <b> <center> + 
# <b> <Center> Install <img src="https://www.libtorrent.org/img/logo-color-text.png" height="20px" class="icon" alt="Icon">

In [ ]:
#@title <b>↓↓ Run the Following Cell
 
from google.colab import drive
drive.mount("/content/drive")

import IPython.display as display
!pip install --quiet --disable-pip-version-check libtorrent
display.display(display.Markdown("### libtorrent installation sucessful!"))
display.clear_output(wait=True)

___________________________________________

#<b><center> Enter Path📁 
#<b><center>&
#<b><center>Link or Magnet🧲

In [ ]:

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import libtorrent as lt
import time
from tqdm.auto import tqdm
import requests
import tempfile
import os

ses = lt.session()
ses.listen_on(6881, 6891)
ses.start_dht()  # Enable DHT
ses.set_max_connections(100)  # Set connection limit to 100

downloads = []

download_path = "/content/X"  #@param {type:"string"}
params = {
    "save_path": download_path,
    "storage_mode": lt.storage_mode_t(2)
}

link = "magnet:?xt=urn:enteryourmagnethere" #@param {type:"string"}

if "magnet:" in link:
    handle = lt.add_magnet_uri(ses, link, params)
else:
    # Download the .torrent file from the URL
    response = requests.get(link)
    temp_file_path = "/content/temp.torrent"  # Set the temporary file path in Colab
    with open(temp_file_path, 'wb') as temp_file:
        temp_file.write(response.content)
    with open(temp_file_path, 'rb') as file:
        data = file.read()
        info = lt.torrent_info(lt.bdecode(data))
    handle = lt.add_magnet_uri(ses, lt.make_magnet_uri(info), params)
    os.remove(temp_file_path)

print("Downloading metadata...")

while not handle.has_metadata():
    time.sleep(1)

print("Starting", handle.name())

# Get initial peer list
peers = handle.get_peer_info()

with tqdm(total=handle.status().total_wanted, unit='B', unit_scale=True, desc=handle.name()) as pbar:
    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata',
                     'downloading', 'finished', 'seeding', 'allocating']
        pbar.set_postfix({'State': state_str[s.state], 'Seeding': s.num_complete, 'Leeching': s.num_incomplete})
        downloaded = s.total_wanted_done
        pbar.update(downloaded - pbar.n)
        pbar.set_description(f"{handle.name()} - {s.download_rate/1000:.1f} KB/s - {s.progress*100:.2f}%")

        # Check and switch to a seeder with higher download speed
        max_speed = 0
        max_speed_peer = None
        for peer in peers:
            download_rate = (peer.total_download - peer.total_upload) / (time.time() - peer.last_active)
            if download_rate > max_speed:
                max_speed = download_rate
                max_speed_peer = peer

        if max_speed_peer:
            endpoint = max_speed_peer.ip
            handle.connect_peer(endpoint)

        time.sleep(1)

        # Update peer list
        peers = handle.get_peer_info()

print("Download Completed!")